In [ ]:
# -*- coding: utf-8 -*-
"""app7.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1JahsClVeKW4dXh8wfuPYhSOPuUbQRXaG
"""

import pandas as pd
import click
from transformers import pipeline
from diffusers import StableDiffusionPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Load necessary models
text_generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")

# Load datasets
users = pd.read_csv("users_data.csv")
transactions = pd.read_csv("transactions_data.csv")

# Preprocess transactions
transactions['date'] = pd.to_datetime(transactions['date'])
transactions['amount'] = transactions['amount'].replace('[\$,]', '', regex=True).astype(float)

# Feature 1: Total monthly spend
transactions['month'] = transactions['date'].dt.month
monthly_spend = transactions.groupby(['client_id', 'month'])['amount'].sum().reset_index(name='monthly_spend')

# Feature 2: Average transaction amount
avg_transaction_amount = transactions.groupby('client_id')['amount'].mean().reset_index(name='avg_transaction_amount')

# Feature 3: Chip transaction percentage
chip_usage = transactions.groupby('client_id')['use_chip'].apply(lambda x: (x == 'Chip Transaction').mean()).reset_index(name='chip_transaction_pct')

# Feature 4: Top merchant category
top_merchant_category = transactions.groupby('client_id')['merchant_category'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index(name='top_merchant_category')

# Preprocess user data
users['per_capita_income'] = users['per_capita_income'].replace('[\$,]', '', regex=True).astype(float)
users['yearly_income'] = users['yearly_income'].replace('[\$,]', '', regex=True).astype(float)
users['total_debt'] = users['total_debt'].replace('[\$,]', '', regex=True).astype(float)

# Feature 5: Debt-to-income ratio
users['debt_to_income_ratio'] = users['total_debt'] / users['yearly_income']

# Feature 6: Discretionary income
users['discretionary_income'] = users['yearly_income'] - users['total_debt']

# Feature 7: Credit score category
def credit_score_category(score):
    if score >= 750:
        return "Excellent"
    elif score >= 700:
        return "Good"
    elif score >= 650:
        return "Fair"
    else:
        return "Poor"

users['credit_score_category'] = users['credit_score'].apply(credit_score_category)

# Merge datasets
customer_features = pd.merge(monthly_spend, avg_transaction_amount, on='client_id')
customer_features = pd.merge(customer_features, chip_usage, on='client_id')
customer_features = pd.merge(customer_features, top_merchant_category, on='client_id')
final_data = pd.merge(customer_features, users, left_on='client_id', right_on='id', how='left')

# Clustering
features = ['monthly_spend', 'avg_transaction_amount', 'chip_transaction_pct', 'debt_to_income_ratio']
X = final_data[features].fillna(0)  # Handle missing values by filling with 0

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Determine optimal number of clusters (optional)
silhouette_scores = {}
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    silhouette_scores[k] = silhouette_score(X_scaled, kmeans.labels_)

# Choose k (e.g., based on silhouette score)
optimal_k = max(silhouette_scores, key=silhouette_scores.get)

# Clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
final_data['cluster'] = kmeans.fit_predict(X_scaled)

# Map cluster values to descriptive profile names
cluster_profile_map = {
    0: "Budget-Conscious Shoppers",
    1: "Medium Range Spenders",
    2: "Luxury Spenders"
}

# Create a new column 'cluster_profile' based on the mapping
final_data['cluster_profile'] = final_data['cluster'].map(cluster_profile_map)

# Function to generate email content with sample offers
def generate_email_prompt(client_data):
    category = client_data['cluster_profile']

    if category == "Budget-Conscious Shoppers":
        offer_prompt = """
        Generate three unique, enticing offers for a credit card user in the 'Budget-Conscious Shoppers' category.
        Focus on saving money on everyday expenses like groceries, fuel, and utility payments.
        Sample Offers:
        1. **Cashback Offer**: 5% cashback on all grocery purchases for the next 3 months.
        2. **Fuel Discount**: Get a 10% discount on fuel purchases at select petrol stations.
        3. **Utility Payment Waiver**: Waive the monthly service fee for utility bill payments for the first 3 months.
        """
    elif category == "Medium Range Spenders":
        offer_prompt = """
        Generate three unique, enticing offers for a credit card user in the 'Medium Range Spenders' category.
        Focus on balanced spending habits with moderate rewards for dining, shopping, and entertainment.
        Sample Offers:
        1. **Dining Rewards**: Earn 3x points on dining and food delivery services for the next 2 months.
        2. **Shopping Discounts**: Get 10% off on purchases at major retail stores.
        3. **Seasonal Bonus**: Earn 500 bonus reward points during the festive season for any purchases.
        """
    elif category == "Luxury Spenders":
        offer_prompt = """
        Generate three unique, enticing offers for a credit card user in the 'Luxury Spenders' category.
        Focus on exclusive benefits like premium rewards, concierge services, and luxury perks.
        Sample Offers:
        1. **VIP Lounge Access**: Enjoy complimentary access to airport VIP lounges worldwide for the next 12 months.
        2. **Luxury Purchase Bonus**: Get 10,000 bonus points on your next luxury purchase over $500.
        3. **Exclusive Concierge Service**: 24/7 concierge service for your travel, dining, and event needs.
        """
    else:
        offer_prompt = "Generate three generic credit card offers for a customer with undefined spending habits."

    # Generate offers using the text generator
    offers = text_generator(offer_prompt, max_length=500, num_return_sequences=1)[0]['generated_text']

    # Generate personalized email content
    prompt = f"""
    Write a personalized marketing email for a customer with the following profile:

    **Customer Profile Summary**
    - **Client ID**: {client_data['client_id']}
    - **Age**: {client_data['current_age']}
    - **Credit Score**: {client_data['credit_score']} ({client_data['credit_score_category']})
    - **Monthly Spend**: ${client_data['monthly_spend']}
    - **Average Transaction Amount**: ${client_data['avg_transaction_amount']}
    - **Top Merchant Category**: {client_data['top_merchant_category']}
    - **Financial Focus**: {'Maximizing rewards and savings' if category == 'Budget-Conscious Shoppers' else 'Balancing rewards and convenience' if category == 'Medium Range Spenders' else 'Exclusive premium benefits'}

    Craft an email based on their category: '{category}'.

    Email Template:

    **Subject Line:** Hello {client_data['client_id']}, exciting new offers tailored for you!

    **Email Body:**
    Hello {client_data['client_id']},

    Thanks for using our credit card services! We are thrilled to introduce exclusive offers tailored to your financial profile:

    **Your Profile Highlights:**
    - Monthly Spend: ${client_data['monthly_spend']}
    - Average Transaction Amount: ${client_data['avg_transaction_amount']}
    - Top Merchant Category: {client_data['top_merchant_category']}

    **Exclusive Offers for You ({category}):**
    {offers}

    Keep shopping with us to make the most of these exclusive offers and rewards!

    Thanks and Regards,
    [Dummy Bank]
    """
    return prompt

def get_email_for_client(client_id, final_data):
    client_id = str(client_id) if final_data['client_id'].dtype == 'object' else int(client_id)

    client_data = final_data[final_data['client_id'] == client_id]
    if client_data.empty:
        return f"No data found for Client ID: {client_id}"

    client_data = client_data.iloc[0]  # Safely access the first matching record
    prompt = generate_email_prompt(client_data)
    email_content = text_generator(prompt, max_length=1000, num_return_sequences=1)[0]['generated_text']
    return email_content


# Function to generate a tagline
def generate_tagline(profile_name):
    prompt = f"Generate an attractive, simple credit card essential needs tagline for {profile_name}."
    return text_generator(prompt, max_length=30)[0]['generated_text']

# Function to generate a category-based image
def generate_category_image(profile_name):
    image = pipe(profile_name).images[0]
    image.show()

# CLI command for generating marketing materials
@click.command()
@click.argument('client_id', type=int)
def generate_marketing_material(client_id):
    # Generate the email content
    email_content = get_email_for_client(client_id, final_data)
    print(f"Generated Email for Client ID {client_id}:\n")
    print(email_content)

    # Retrieve client data safely
    client_data = final_data[final_data['client_id'] == client_id]
    if client_data.empty:
        print(f"No data found for Client ID: {client_id}")
        return

    client_data = client_data.iloc[0]  # Safely access the first matching record
    profile_name = client_data['cluster_profile']

    # Generate the tagline
    tagline = generate_tagline(profile_name)
    print(f"\nGenerated Tagline:\n{tagline}")

    # Generate the category-based image
    generate_category_image(profile_name)

if __name__ == '__main__':
    generate_marketing_material()